#### @prompt example

In [ ]:
from openbb import obb
import os
from magentic import prompt
from openbb import obb
obb.account.login(pat=os.environ['PAT_KEY'])

## Six Step Dance via Langchain
1. Taking into Account Company Industry (Growth / Value)
    1. Growth / Value
    2. Sector outperforming market or not
    3. Company outperforming sector
    4. PE vs Industry
2. Determining Company Financial Stability
    1. Debt To Equity
    2. Current Ratio
3. Looking at Historical Earning and Growth
    1. Sales Growth
    2. EPS Growth
    3. Return on Invested Capital
    4. Profit Margin
    5. Cashflow Growth
    6. Equity Growth
    7. Return on Equity
4. Understanding Earnings and Sales Expectations
    1. Analyst Estimates
    2. PEG
5. Checking out competition
    1. P/E
    2. P/S
    3.MOAT
6. Estimating Company Values
    1. Insider Buying
    2. Senate TRading
    3. 13f

## Buffett Criteria
### Gross Margin Indicates profitability > 40% GrossProfit/Revenue GROSS PM close to 40 for recent years
### SG&A Margin efficiency in managing overhead costs < 30% SG&A Expenses / Gross Profit
### R&D Margin r&d spending relative to profitability < 30% R&D Expenses / Gross Profit
### Interest Expense Margin reliance on debt and financial health < 15 % Interest Expense / Operating Income
### Income Tax Margin Gauges tax efficiency ~ 20% Income Tax / Pre-Tax Income
### Profit Margin Shows overall profitabilty relative to revenue > 20% Net Income / Revenue
### EPS Growth Indicates growth and consistency in earnings >0 and growing Net Income / Shares Outstanding




In [2]:
obb.news.world(provider='fmp').to_llm()

'[{"date":"2025-03-18T20:14:00","title":"12 Quality Growth Stocks to Buy for an Unstable Market","images":[{"o":"https:\\/\\/images.financialmodelingprep.com\\/news\\/12-quality-growth-stocks-to-buy-for-an-unstable-20250318.jpg"}],"text":"The trick for investors is to find stocks that are already fairly cheap, have high growth potential, and are in a strong enough financial position to withstand a recession.","url":"https:\\/\\/www.barrons.com\\/articles\\/growth-stocks-buy-falling-market-a68ab02e","site":"barrons.com"},{"date":"2025-03-18T19:48:14","title":"The Fed will update its rate projections Wednesday. Here\'s what to expect","images":[{"o":"https:\\/\\/images.financialmodelingprep.com\\/news\\/the-fed-will-update-its-rate-projections-wednesday-heres-20250318.jpg"}],"text":"Fed officials at this week\'s meeting are expected to hold interest rates steady but adjust their views on the economy and possibly the future path of interest rates. The committee could maintain its outlook 

In [ ]:



@prompt("Explain like I'm five this financial concept: {concept}")
def explain(concept: str) -> str: ...


explain("Subprime mortgage crisis")

"Alright! Imagine you have a friend who really wants a big cookie, but they don't have enough money to buy it. So they borrow money from another friend. But here's the thing: your first friend isn't very good at saving or paying back money, kind of like when you borrow a toy and forget to give it back.\n\nNow, if lots of people who weren't very good at paying back money borrowed to buy cookies, and they couldn't pay the money back, it becomes a big problem. The friends who lent the money might get worried and not have enough to share, and soon there's a big mess because the cookies (or in real life, houses) are not being paid for.\n\nThe subprime mortgage crisis was like this, but with houses and banks instead of cookies and friends. Lots of people borrowed money to buy houses, but then they couldn't pay it back, which caused a lot of trouble for the banks and the whole money system, leading to a big, worldwide financial mess!"

In [ ]:
from magentic import prompt
from pydantic import BaseModel


class Portfolio(BaseModel):
    equity_etf_pct: float
    bond_etf_pc: float
    crypto_etf_pc: float
    commodities_pc: float
    reasoning: str


@prompt("Create a strong portfolio of {size} allocation size.")
def create_portfolio(size: str) -> Portfolio: ...


portfolio = create_portfolio("$50,000")

In [ ]:
print(portfolio)

#### @chatprompt decorator

In [17]:
from magentic import chatprompt, AssistantMessage, SystemMessage, UserMessage
from pydantic import BaseModel

class Quote(BaseModel):
    quote: str
    person: str


@chatprompt(
    SystemMessage("You are an avid reader of financial literature."),
    UserMessage("What is your favorite quote from Warren Buffet?"),
    AssistantMessage(
        Quote(
            quote="Price is what you pay; value is what you get.",
            person="Warren Buffet",
        )
    ),
    UserMessage("What is your favorite quote from {person}?"),
)
def get_finance_quote(person: str) -> Quote: ...


get_finance_quote("Charlie Munger")

Quote(quote='The big money is not in the buying and selling...but in the waiting.', person='Charlie Munger')

#### Function calling

In [ ]:
import os
import requests
from magentic import prompt, FunctionCall
import logging
from openbb import obb
from langchain.tools import Tool
import reticker

def get_income_statement(query:str) -> str:
    try:
        ticker = reticker(query)
        period = 'annual' if 'annual' in query else 'quarterly'
        return obb.equity.fundamental.income(symbol=ticker, period=period, limit=5, provider='fmp').to_llm()
    except Exception as e:
        return f'Unable to find data for {ticker}'

def get_balance_sheet(query:str) -> str:
    try:
        ticker = reticker(query)
        period = 'annual' if 'annual' in query else 'quarterly'
        return obb.equity.fundamental.balance(symbol=ticker, period=period, limit=5, provider='intrinio')
    except Exception as e:
        return f'Unable to find data for {ticker}'


def get_aggressive_smallcaps() -> list :
    return obb.equity.discovery.aggressive_small_caps(sort='desc').to_llm()

def get_sectors_performance() -> list:
    return obb.equity.compare.groups(group='sector', metric='performance', provider='finviz').to_llm()

def get_undervalued_growth() -> list:
    return obb.equity.discovery.undervalued_growth(provider='yfinance').to_llm()


def get_ticker_from_query(query):
    extractor = reticker.TickerExtractor(deduplicate=True)
    tickers = extractor.extract(query)
    if len(tickers) > 1:
        return ','.join(tickers)
    return tickers[0] 


def get_stock_price(query: str) -> str:
    """Get the current stock price for a given ticker."""
    try:
        ticker = get_ticker_from_query(query)
        data = obb.equity.price.quote(symbol=ticker)

        return data.to_llm()
    except Exception as e:
        return f"Error fetching stock price for {ticker}: {str(e)}"

def get_company_overview(query: str) -> str:
    """Get an overview of a company for a given ticker."""
    try:
        ticker = get_ticker_from_query(query)
        data = obb.equity.profile(symbol=ticker)
        return data.to_llm()
    except Exception as e:
        return f"Error fetching company overview for {ticker}: {str(e)}"

def get_latest_news_for_company(query : str) -> str:
    """ Get latest news for a company """
    try:
        ticker = get_ticker_from_query(query)
        return obb.news.company(symbol=ticker)
    except Exception as e:
        return f"Error fetching stock price for {ticker}: {str(e)}"

@prompt(
    "Use the appropriate search function to answer: {question}",
    functions=[get_stock_price, get_aggressive_smallcaps, get_sectors_performance, get_undervalued_growth, get_latest_news_for_company,
               get_balance_sheet, get_income_statement],
)
def perform_search(question: str) -> FunctionCall[str]: ...


output = perform_search("Generate a table out of the data for aggressive smallcaps?")
output()

'[{"symbol":"PHAT","name":"Phathom Pharmaceuticals, Inc.","price":5.54,"change":0.6999998,"percent_change":0.14462805,"volume":2382117,"open":4.99,"high":5.82,"low":4.98,"previous_close":4.84,"ma50":6.0248,"ma200":11.11375,"year_high":19.71,"year_low":4.07,"market_cap":385788992.0,"shares_outstanding":69637000.0,"book_value":-3.701,"price_to_book":-1.4968927,"eps_ttm":-5.29,"eps_forward":-4.0,"pe_forward":-1.385,"dividend_yield":0.0,"exchange":"NMS","exchange_timezone":"America\\/New_York","earnings_date":"2025-03-06T13:00:02Z","currency":"USD"},{"symbol":"PHAT","name":"Phathom Pharmaceuticals, Inc.","price":5.54,"change":0.6999998,"percent_change":0.14462805,"volume":2382117,"open":4.99,"high":5.82,"low":4.98,"previous_close":4.84,"ma50":6.0248,"ma200":11.11375,"year_high":19.71,"year_low":4.07,"market_cap":385788992.0,"shares_outstanding":69637000.0,"book_value":-3.701,"price_to_book":-1.4968927,"eps_ttm":-5.29,"eps_forward":-4.0,"pe_forward":-1.385,"dividend_yield":0.0,"exchange":"N

In [10]:
output()

'{"id":"067d91e3-7273-740d-8000-c29f742410fb","results":[{"symbol":"AAPL","asset_type":null,"name":"Apple Inc.","exchange":"NASDAQ","bid":null,"bid_size":null,"bid_exchange":null,"ask":null,"ask_size":null,"ask_exchange":null,"quote_conditions":null,"quote_indicators":null,"sales_conditions":null,"sequence_number":null,"market_center":null,"participant_timestamp":null,"trf_timestamp":null,"sip_timestamp":null,"last_price":214.0,"last_tick":null,"last_size":null,"last_timestamp":"2025-03-17T20:00:01Z","open":213.36,"high":215.22,"low":209.98,"close":null,"volume":47841607,"exchange_volume":null,"prev_close":213.49,"change":0.51,"change_percent":0.0023889,"year_high":260.1,"year_low":164.08,"price_avg50":234.2014,"price_avg200":227.89275,"avg_volume":53441279,"market_cap":3214729400000.0,"shares_outstanding":15022100000,"eps":6.97,"pe":30.7,"earnings_announcement":"2025-04-30T10:59:00Z"}],"provider":"fmp","warnings":null,"chart":null,"extra":{"metadata":{"arguments":{"provider_choices":{

In [5]:
output = perform_search('Please give me a list of latest aggressive smallcaps')
output()

'[{"symbol":"PHAT","name":"Phathom Pharmaceuticals, Inc.","price":5.54,"change":0.6999998,"percent_change":0.14462805,"volume":2382117,"open":4.99,"high":5.82,"low":4.98,"previous_close":4.84,"ma50":6.0248,"ma200":11.11375,"year_high":19.71,"year_low":4.07,"market_cap":385788992.0,"shares_outstanding":69637000.0,"book_value":-3.701,"price_to_book":-1.4968927,"eps_ttm":-5.29,"eps_forward":-4.0,"pe_forward":-1.385,"dividend_yield":0.0,"exchange":"NMS","exchange_timezone":"America\\/New_York","earnings_date":"2025-03-06T13:00:02Z","currency":"USD"},{"symbol":"KVHI","name":"KVH Industries, Inc.","price":5.26,"change":0.44000006,"percent_change":0.09128632,"volume":38795,"open":4.85,"high":5.4,"low":4.85,"previous_close":4.82,"ma50":5.7774,"ma200":5.08275,"year_high":6.16,"year_low":4.17,"market_cap":103958648.0,"shares_outstanding":19764000.0,"book_value":7.007,"price_to_book":0.75067794,"eps_ttm":-0.57,"eps_forward":-0.12,"pe_forward":-43.833336,"dividend_yield":0.0,"exchange":"NMS","exch

In [ ]:
output().to_df()

#### Prompt Chains

In [ ]:
import csv
from magentic import prompt_chain


def get_earnings_calendar(ticker: str, api_key: str = FMP_KEY) -> list:
    url = f"https://www.alphavantage.co/query?function=EARNINGS_CALENDAR&symbol={ticker}&horizon=12month&apikey={api_key}"
    with requests.Session() as s:
        download = s.get(url)
        decoded_content = download.content.decode("utf-8")
        cr = csv.reader(decoded_content.splitlines(), delimiter=",")
        my_list = list(cr)
    return my_list


@prompt_chain(
    "What's {ticker} expected earnings dates for the next 12 months?",
    functions=[get_earnings_calendar],
)
def get_earnings(ticker: str) -> str: ...


get_earnings("IBM")

#### Streaming Response

In [ ]:
from magentic import StreamedStr


@prompt("Explain to me {term} in a way a 5-year-old would understand.")
def describe_finance_term(term: str) -> StreamedStr: ...


# Print the chunks while they are being received
for chunk in describe_finance_term("liquidity"):
    print(chunk, end="")

#### Streaming Structured Outputs

In [ ]:
from collections.abc import Iterable
from time import time


class Portfolio(BaseModel):
    equity_etf_pct: float
    bond_etf_pc: float
    crypto_etf_pc: float
    commodities_pc: float
    reasoning: str


@prompt("Create {n_portfolio} portfolios with varying deegress of risk apetite.")
def create_portfolios(n_portfolio: int) -> Iterable[Portfolio]: ...


start_time = time()
for portfolio in create_portfolios(3):
    print(f"{time() - start_time:.2f}s : {portfolio}")

#### Asynchronous Streaming

In [ ]:
import asyncio
from typing import AsyncIterable


@prompt("List three high-growth stocks.")
async def iter_growth_stocks() -> AsyncIterable[str]: ...


@prompt("Tell me more about {stock_symbol}")
async def tell_me_more_about(stock_symbol: str) -> str: ...


start_time = time()
tasks = []
async for stock in await iter_growth_stocks():
    # Use asyncio.create_task to schedule the coroutine for execution before awaiting it
    # This way descriptions will start being generated while the list of stocks is still being generated
    task = asyncio.create_task(tell_me_more_about(stock))
    tasks.append(task)

descriptions = await asyncio.gather(*tasks)

for desc in descriptions:
    print(desc)